In [15]:
import asyncio
import requests
import re
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [16]:
apiKey = ''
apiSecret = ''
passPhrase = ''

In [17]:
#Granularity 
##60, 300, 900, 3600, 21600, 86400 === correspond to timeslices represent one minute, five minutes, fifteen minutes, one hour, six hours, and one day, respectively.

def cbpGetHistoricRates(market='ETH-USD', granularity=900, iso8601start='', iso8601end=''):
    
    p = re.compile(r"^[A-Z]{3,4}\-[A-Z]{3,4}$")
    if not p.match(market):
        raise TypeError('Coinabse Pro market required.')
        
    if not isinstance(granularity, int):
        raise TypeError('Granularity integer required.')
        
    if not granularity in [60, 300, 900, 3600, 21600, 86400]:
        raise TypeError('Granularity options: 60, 300, 900, 3600, 21600, 86400')
        
    if not isinstance(iso8601start, str):
        raise TypeError('ISO8601 start integer is string required')
        
    if not isinstance(iso8601end, str):
        raise TypeError('ISO8601 end integer is string required')
        
    api = 'https://api.pro.coinbase.com/products/' + market + '/candles?granularity=' + str(granularity) + '&start=' + iso8601start + '&end=' + iso8601end
    resp = requests.get(api, verify=False, timeout=5)
    if resp.status_code != 200:
        raise Exception('Get' + api + '{}' .format(resp.status_code))
        
    df = pd.DataFrame(resp.json(), columns = ['iso8601','low','high','open','close','volume'])
    df = df.iloc[::-1].reset_index()
    
    for index, row in df.iterrows():
        iso8601 = datetime.fromtimestamp(row['iso8601'])
        df.at[index, 'datetime'] = datetime.strftime(iso8601, "%Y/%m/%d %H:%M:%S")
        
    df = df.reindex(columns = ['iso8601','datetime','low','high','open','close','volume'])
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    return df

In [18]:
data = cbpGetHistoricRates()

C:\Users\jwkon\anaconda3\envs\tensor\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pro.coinbase.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [19]:
data

,iso8601,datetime,low,high,open,close,volume
0,1610005500,2021-01-07 02:45:00,1182.95,1194.59,1193.87,1187.99,4581.701713
1,1610006400,2021-01-07 03:00:00,1188.50,1194.59,1188.50,1189.07,2847.584294
2,1610007300,2021-01-07 03:15:00,1160.71,1189.08,1189.08,1179.86,10857.607905
3,1610008200,2021-01-07 03:30:00,1176.22,1195.76,1179.85,1195.76,3767.329980
4,1610009100,2021-01-07 03:45:00,1192.08,1201.86,1195.41,1197.33,3764.195595
...,...,...,...,...,...,...,...
295,1610271000,2021-01-10 04:30:00,1336.03,1344.75,1337.41,1340.05,1504.440310
296,1610271900,2021-01-10 04:45:00,1332.56,1348.57,1340.06,1333.96,3224.166264
297,1610272800,2021-01-10 05:00:00,1329.30,1341.97,1334.29,1337.90,3066.177716
298,1610273700,2021-01-10 05:15:00,1314.28,1341.86,1338.05,1318.94,5359.582210


In [20]:
data = data.drop(['iso8601'], axis=1)
data['date'] = pd.to_datetime(data['datetime']).dt.date
#data = data.set_index(pd.DatetimeIndex(data['date'].values))
data = data.sort_index()
data = data.drop(columns=['datetime'])
data.head()

,low,high,open,close,volume,date
0,1182.95,1194.59,1193.87,1187.99,4581.701713,2021-01-07
1,1188.50,1194.59,1188.50,1189.07,2847.584294,2021-01-07
2,1160.71,1189.08,1189.08,1179.86,10857.607905,2021-01-07
3,1176.22,1195.76,1179.85,1195.76,3767.329980,2021-01-07
4,1192.08,1201.86,1195.41,1197.33,3764.195595,2021-01-07


In [21]:
print(min(data['date']))
print(max(data['date']))
#Year-Day-Month#

2021-01-07
2021-01-10


In [22]:
data.to_csv('C:\\Users\jwkon\Data Learning\Investiment Predictions and Algorithms\Raw_Data(PulledData)\EtherJan7Jan10.csv', index=False)